# **Proyek Sistem Rekomendasi Buku**

## Import Libraries

In [ ]:
# Import library dasar
import pandas as pd
import numpy as np

# Import library untuk visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

# Import library untuk machine learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Import library untuk interaktivitas di notebook
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

print("Semua library berhasil diimpor.")

## Data Loading

In [ ]:
# Memuat dataset dari file CSV
try:
    books = pd.read_csv('./datasets/Books.csv', on_bad_lines='skip', encoding='latin-1', low_memory=False)
    ratings = pd.read_csv('./datasets/Ratings.csv', on_bad_lines='skip', encoding='latin-1', low_memory=False)
    users = pd.read_csv('./datasets/Users.csv', on_bad_lines='skip', encoding='latin-1', low_memory=False)
    print("Dataset berhasil dimuat.")
except FileNotFoundError as e:
    print(f"Error: {e}.")
    print("Pastikan file Books.csv, Ratings.csv, dan Users.csv berada di direktori yang sama.")

## Data Understanding - Part 1

### Inspeksi Data Awal

In [ ]:
# Inspeksi Data Awal (Data Understanding - Part 1)
print("--- Inspeksi DataFrame 'books' ---")
display(books.head())
print("\nInfo 'books':")
books.info()
print("\nCek Missing Values 'books':")
print(books.isnull().sum())
print("\n" + "="*40 + "\n")

print("--- Inspeksi DataFrame 'users' ---")
display(users.head())
print("\nInfo 'users':")
users.info()
print("\nCek Missing Values 'users':")
print(users.isnull().sum())
print("\n" + "="*40 + "\n")

print("--- Inspeksi DataFrame 'ratings' ---")
display(ratings.head())
print("\nInfo 'ratings':")
ratings.info()
print("\nCek Missing Values 'ratings':")
print(ratings.isnull().sum())

## Data Preparation - Part 1

### Data Cleaning & Preprocessing

In [ ]:
# Data Cleaning & Preprocessing (Data Preparation - Part 1)
print("--- Memulai Data Cleaning ---")

# Membersihkan kolom 'Year-Of-Publication'
# Mengubah ke numerik, error diubah menjadi NaN (Not a Number)
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'], errors='coerce')
# Mengganti tahun yang tidak valid (misal: 0 atau > 2025) dengan NaN
books.loc[(books['Year-Of-Publication'] == 0) | (books['Year-Of-Publication'] > 2025), 'Year-Of-Publication'] = np.NAN
# Mengisi NaN dengan median tahun publikasi
books['Year-Of-Publication'].fillna(round(books['Year-Of-Publication'].median()), inplace=True)

# Membersihkan kolom 'Publisher' dan 'Book-Author'
# Mengisi NaN dengan 'Unknown'
books['Publisher'].fillna('Unknown', inplace=True)
books['Book-Author'].fillna('Unknown', inplace=True)

# Membersihkan kolom 'Age' di 'users'
# Mengganti umur yang tidak realistis (< 5 atau > 100) dengan NaN
users.loc[(users['Age'] < 5) | (users['Age'] > 100), 'Age'] = np.nan
# Mengisi NaN dengan median umur
users['Age'].fillna(users['Age'].median(), inplace=True)


print("--- Data Cleaning Selesai ---")
print("\nInfo 'books' setelah cleaning:")
books.info()
print("\nInfo 'users' setelah cleaning:")
users.info()

## Data Understanding - Part 2

### Exploratory Data Analysis (EDA)

In [ ]:
# Exploratory Data Analysis (EDA) - (Data Understanding - Part 2)

# --- Univariate Analysis ---
# 1. Distribusi Umur Pengguna
plt.figure(figsize=(10, 6))
sns.histplot(users['Age'], bins=30, kde=True)
plt.title('Distribusi Umur Pengguna')
plt.xlabel('Umur')
plt.ylabel('Jumlah Pengguna')
plt.show()

# 2. Distribusi Tahun Publikasi Buku
plt.figure(figsize=(10, 6))
sns.histplot(books['Year-Of-Publication'], bins=30, kde=True)
plt.title('Distribusi Tahun Publikasi Buku')
plt.xlabel('Tahun Publikasi')
plt.ylabel('Jumlah Buku')
plt.show()

# --- Bivariate Analysis ---
# Gabungkan semua data untuk analisis mendalam
df_merged = pd.merge(ratings, users, on='User-ID')
df_merged = pd.merge(df_merged, books, on='ISBN')

# 3. Rata-rata Rating berdasarkan Kelompok Umur
# Membuat kategori umur
bins = [0, 18, 30, 50, 100]
labels = ['<18', '18-30', '31-50', '>50']
df_merged['Age-Group'] = pd.cut(df_merged['Age'], bins=bins, labels=labels, right=False)

# Mengabaikan rating 0 (implisit) untuk analisis ini
avg_rating_by_age = df_merged[df_merged['Book-Rating'] != 0].groupby('Age-Group')['Book-Rating'].mean()

plt.figure(figsize=(10, 6))
avg_rating_by_age.plot(kind='bar', color=sns.color_palette('viridis'))
plt.title('Rata-rata Rating Buku Berdasarkan Kelompok Umur')
plt.xlabel('Kelompok Umur')
plt.ylabel('Rata-rata Rating')
plt.xticks(rotation=0)
plt.show()

## Data Preparation - Part 2

### Menggabungkan dan Finalisasi Data

In [ ]:
# Menggabungkan dan Finalisasi Data (Data Preparation - Part 2)

# Menggabungkan ratings dan books
df_final = pd.merge(ratings, books, on='ISBN', how='left')

# Menghapus duplikat berdasarkan interaksi user-judul buku
df_final.drop_duplicates(['User-ID', 'Book-Title'], inplace=True)

# Menghapus baris dimana atribut penting untuk model tidak ada
df_final.dropna(subset=['Book-Title', 'Book-Author'], inplace=True)

print(f"Jumlah data setelah digabung dan dibersihkan: {len(df_final)}")

## Model Development

### Content-Based Filtering

In [ ]:
# Model Development - Content-Based Filtering

# Persiapan data unik buku
books_for_content = df_final.drop_duplicates('Book-Title')[['Book-Title', 'Book-Author']]

# Inisialisasi dan fitting TF-IDF
tf_idf = TfidfVectorizer()
tfidf_matrix = tf_idf.fit_transform(books_for_content['Book-Author'].fillna(''))
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index=books_for_content['Book-Title'], columns=books_for_content['Book-Title'])

# Fungsi rekomendasi
def get_book_recommendations(book_title, k=5):
    try:
        index = cosine_sim_df.loc[:, book_title].to_numpy().argpartition(range(-1, -k, -1))
        closest = cosine_sim_df.columns[index[-1:-(k+2):-1]]
        closest = closest.drop(book_title, errors='ignore')
        recommendations = pd.DataFrame(closest).merge(books_for_content).head(k)
        return recommendations
    except KeyError:
        print(f"Buku dengan judul '{book_title}' tidak ditemukan.")
        return pd.DataFrame()

# --- Widget Interaktif ---
book_titles_list = books_for_content['Book-Title'].sort_values().tolist()

@interact(
    book_title=widgets.Dropdown(options=book_titles_list, description='Pilih Judul Buku:', layout={'width': '500px'}),
    k=widgets.IntSlider(min=5, max=15, step=1, value=10, description='Jumlah Rekomendasi:')
)
def interactive_recommendations(book_title, k):
    recommendations_df = get_book_recommendations(book_title, k)
    if not recommendations_df.empty:
        display(recommendations_df)

### Preparation for Collaborative Filtering

In [ ]:
# --- Preparation for Collaborative Filtering ---

# Menggunakan data yang sudah digabungkan dan bersih
df_collab = df_final.copy()

# Konseptual Outlier Handling:
# Untuk model collaborative, kita fokus pada pengguna yang aktif dan buku yang cukup populer
# Untuk membantu mengurangi noise dan membuat model lebih stabil
user_counts = df_collab['User-ID'].value_counts()
book_counts = df_collab['ISBN'].value_counts()

# Filter pengguna yang memberi rating > 10 buku dan buku yang menerima > 5 rating
df_collab = df_collab[df_collab['User-ID'].isin(user_counts[user_counts > 10].index)]
df_collab = df_collab[df_collab['ISBN'].isin(book_counts[book_counts > 5].index)]

# Fokus pada rating eksplisit (1-10)
df_collab = df_collab[df_collab['Book-Rating'] != 0]

print(f"Jumlah data setelah difilter untuk Collaborative Filtering: {len(df_collab)}")


# Encoding User-ID dan ISBN
user_ids = df_collab['User-ID'].unique().tolist()
user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}

isbn_ids = df_collab['ISBN'].unique().tolist()
isbn_to_isbn_encoded = {x: i for i, x in enumerate(isbn_ids)}
isbn_encoded_to_isbn = {i: x for i, x in enumerate(isbn_ids)}

# Mapping dan Normalisasi
df_collab['user'] = df_collab['User-ID'].map(user_to_user_encoded)
df_collab['book'] = df_collab['ISBN'].map(isbn_to_isbn_encoded)

min_rating = min(df_collab['Book-Rating'])
max_rating = max(df_collab['Book-Rating'])
df_collab['rating'] = df_collab['Book-Rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values

# Membagi data training dan validasi
df_shuffled = df_collab.sample(frac=1, random_state=42)
x = df_shuffled[['user', 'book']].values
y = df_shuffled['rating'].values

train_indices = int(0.8 * df_shuffled.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)

### Collaborative Filtering

In [ ]:
# Model Development - Collaborative Filtering
class RecommenderNet(tf.keras.Model):
    def __init__(self, num_users, num_books, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_books = num_books
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(num_users, embedding_size, embeddings_initializer='he_normal', embeddings_regularizer=keras.regularizers.l2(1e-6))
        self.user_bias = layers.Embedding(num_users, 1)
        self.book_embedding = layers.Embedding(num_books, embedding_size, embeddings_initializer='he_normal', embeddings_regularizer=keras.regularizers.l2(1e-6))
        self.book_bias = layers.Embedding(num_books, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        book_vector = self.book_embedding(inputs[:, 1])
        book_bias = self.book_bias(inputs[:, 1])
        dot_user_book = tf.tensordot(user_vector, book_vector, 2)
        x = dot_user_book + user_bias + book_bias
        return tf.nn.sigmoid(x)

model = RecommenderNet(len(user_to_user_encoded), len(isbn_to_isbn_encoded), 50)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

## Training the Collaborative Model

In [ ]:
# Training the Collaborative Model
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=20,
    validation_data=(x_val, y_val)
)

## Evaluation Plot

In [ ]:
# Plotting hasil training
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('Model Metrics (RMSE)')
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Generating Recommendations from Collaborative Model

In [ ]:
# Generating Recommendations from Collaborative Model

def get_collab_recommendations(user_id_sample):
    books_read_by_user = df_collab[df_collab['User-ID'] == user_id_sample]
    books_not_read = books[~books['ISBN'].isin(books_read_by_user['ISBN'].values)]['ISBN']
    
    books_not_read_encoded_keys = set(isbn_to_isbn_encoded.keys())
    books_not_read = list(set(books_not_read).intersection(books_not_read_encoded_keys))
    
    if not books_not_read:
        print(f"Tidak dapat menemukan buku baru untuk direkomendasikan kepada User {user_id_sample}.")
        return

    books_not_read_encoded = [[isbn_to_isbn_encoded.get(x)] for x in books_not_read]
    user_encoder = user_to_user_encoded.get(user_id_sample)
    
    user_book_array = np.hstack(([[user_encoder]] * len(books_not_read_encoded), books_not_read_encoded))
    predicted_ratings = model.predict(user_book_array, verbose=0).flatten()
    
    top_ratings_indices = predicted_ratings.argsort()[-10:][::-1]
    recommended_isbn_ids = [isbn_encoded_to_isbn.get(books_not_read_encoded[x][0]) for x in top_ratings_indices]
    
    print("=" * 40)
    print(f"Menampilkan rekomendasi untuk User ID: {user_id_sample}")
    print("=" * 40)
    
    print("\nBuku yang pernah diberi rating tinggi oleh user ini:")
    top_books_user = books_read_by_user.sort_values(by='Book-Rating', ascending=False).head(5)
    top_books_df = pd.merge(top_books_user, books, on='ISBN')[['Book-Title', 'Book-Rating']]
    if not top_books_df.empty:
        display(top_books_df)
    else:
        print("User ini belum memberikan rating tinggi pada buku manapun.")

    print("\nTop 10 Rekomendasi Buku:")
    recommended_books = books[books['ISBN'].isin(recommended_isbn_ids)][['Book-Title', 'Book-Author']]
    display(recommended_books)


# --- Widget Interaktif ---
user_ids_list_collab = df_collab['User-ID'].unique().tolist()
user_ids_list_collab.sort()

@interact(user_id=widgets.Dropdown(options=user_ids_list_collab, description='Pilih User ID:'))
def interactive_collab_recommendations(user_id):
    get_collab_recommendations(user_id)